In [1]:
from euroleague_api.game_stats import GameStats
from euroleague_api.boxscore_data  import BoxScoreData
from dotenv import load_dotenv
import os
from supabase import create_client
import fonctions.fonctions_api as f

C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bs = BoxScoreData()
gs = GameStats()

load_dotenv()  # charge le .env
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
season = os.getenv("SEASON")
supabase = create_client(url, key)

game_code = 1


In [4]:
match_ids_season = f.get_match_ids_par_saison(supabase, season)

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Match?select=id_match&season=eq.2024 "HTTP/2 200 OK"


In [8]:
not_yet = list(set([i for i in range(1,334)]) - set(match_ids_season))

In [ ]:
for game_code in not_yet:
    print(f"\n🔄 Traitement du match {game_code}...")
    try:
        # 🔹 Récupération des données du match
        ggs = gs.get_game_report(season=season, game_code=game_code)
        boxscore = bs.get_player_boxscore_stats_data(season=season, gamecode=game_code)
        
        # 🔹 Formatage de la date
        date = ggs["date"].to_list()[0]
        
        # 🔹 Nettoyage du boxscore
        boxscore = boxscore[
            (boxscore['Minutes'] != "DNP") &
            (boxscore["Player_ID"] != "Team") &
            (boxscore["Player_ID"] != "Total")
        ][["Player_ID", "Player", "Team", "Valuation"]].reset_index(drop=True)

        # 🔹 Équipes
        id_equipe_local = ggs['local.club.code'].to_list()[0]
        nom_local = ggs['local.club.name'].to_list()[0]
        id_equipe_road = ggs['road.club.code'].to_list()[0]
        nom_road = ggs['road.club.name'].to_list()[0]

        # 🔹 Ajout des équipes
        f.ajouter_equipe(supabase, id_equipe_local, nom_local)
        f.ajouter_equipe(supabase, id_equipe_road, nom_road)

        # 🔹 Ajout du match
        f.ajouter_match(
            supabase, game_code, season,
            id_equipe_local, id_equipe_road,
            ggs['local.score'].to_list()[0],
            ggs['road.score'].to_list()[0],
            date
        )

        # 🔹 Traitement de chaque joueur
        for _, row in boxscore.iterrows():
            id_joueur = row["Player_ID"]
            equipe = row["Team"]
            try:
                nom, prenom = row["Player"].split(", ")
            except ValueError:
                print(f"❌ Format incorrect pour Player : {row['Player']} → ignoré")
                continue

            perf = row["Valuation"]

            try:
                f.ajouter_joueur_si_absent(supabase, id_joueur, nom, prenom)
                f.verifier_ou_ajouter_contrat(supabase, id_joueur, equipe, date)
                id_contrat = f.recuperer_id_contrat(supabase, id_joueur, equipe)
                if id_contrat:
                    f.ajouter_performance(supabase, season, game_code, id_contrat, perf)
                else:
                    print(f"⚠️ Aucun contrat actif pour {id_joueur} → {equipe}")
            except Exception as e:
                print(f"❌ Erreur pour le joueur {id_joueur} ({nom} {prenom}) : {e}")
                continue

    except Exception as e:
        print(f"\n❌ Erreur dans game_code {game_code} → {e}")
        continue



🔄 Traitement du match 2...


INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Equipe?select=id_equipe&id_equipe=eq.MCO "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Equipe?select=id_equipe&id_equipe=eq.MIL "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Equipe "HTTP/2 201 Created"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Match?select=id_match&id_match=eq.2&season=eq.2024 "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Match "HTTP/2 201 Created"


ℹ️ Équipe MCO existe déjà, aucune modification.
✅ Équipe MIL - EA7 Emporio Armani Milan ajoutée.


INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur?select=id_joueur&id_joueur=eq.P011212+++ "HTTP/2 200 OK"
INFO:httpx:HTTP Request: DELETE https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur?id_joueur=eq.P011212+++ "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur "HTTP/2 201 Created"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P011212+++&id_equipe=eq.MCO&END=is.null "HTTP/2 200 OK"


✅ Match ajouté : MCO 93 - 80 MIL
✅ Joueur ajouté : ELIE OKOBO (ID P011212   )


...

INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P005161+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P005161+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Performance "HTTP/2 201 Created"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur?select=id_joueur&id_joueur=eq.P011204+++ "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P011204+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"


ℹ️ Contrat actif déjà en place pour P005161    → PAN
⚠️ PER négatif détecté pour contrat 204, ajusté à 0.
✅ Performance ajoutée : contrat 204, match 98, PER = 0
ℹ️ Joueur déjà existant : Jerian GRANT
ℹ️ Contrat actif déjà en place pour P011204    → PAN


INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P011204+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Performance "HTTP/2 201 Created"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur?select=id_joueur&id_joueur=eq.P012774+++ "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P012774+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P012774+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"


✅ Performance ajoutée : contrat 17, match 98, PER = 7
ℹ️ Joueur déjà existant : Kendrick NUNN
ℹ️ Contrat actif déjà en place pour P012774    → PAN


INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Performance "HTTP/2 201 Created"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Joueur?select=id_joueur&id_joueur=eq.P003842+++ "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P003842+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: GET https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Contrat?select=id_contrat&id_joueur=eq.P003842+++&id_equipe=eq.PAN&END=is.null "HTTP/2 200 OK"
INFO:httpx:HTTP Request: POST https://wfevzenqmhhinyodslro.supabase.co/rest/v1/Performance "HTTP/2 201 Created"
